# Assignment 1

**Group 22**

A. Siganos ( 1283871 )     
J. Gómez Robles ( 1286552 )

In [50]:
import numpy as np
np.random.seed(13) #TODO Check if this is used for sgd
import keras.backend as K
from keras.models import Sequential
from keras.layers import Dense, Embedding, Reshape, Lambda
from keras.utils import np_utils
from keras.utils.data_utils import get_file
from keras.preprocessing.text import Tokenizer
from keras.utils.vis_utils import model_to_dot
from keras.preprocessing import sequence
from gensim.models import KeyedVectors
from sklearn.manifold import TSNE
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.neighbors import NearestNeighbors as nn
from matplotlib import pylab
from __future__ import division

In [58]:
# DO NOT Modify the lines in this cell
path = 'alice.txt'
corpus = open(path).readlines()[0:700]

corpus = [sentence for sentence in corpus if sentence.count(" ") >= 2]

tokenizer = Tokenizer(filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n'+"'")
tokenizer.fit_on_texts(corpus)
corpus = tokenizer.texts_to_sequences(corpus)
nb_samples = sum(len(s) for s in corpus)
V = len(tokenizer.word_index) + 1

# Is this something they need to change?
dim = 100
window_size = 2 #use this window size for Skipgram, CBOW, and the model with the additional hidden layer
window_size_corpus = 4 #use this window size for the co-occurrence matrix

## Question 1

### Co-occurrence Matrix
Use the provided code to load the "Alice in Wonderland" text document. 
1. Implement the word-word co-occurrence matrix for “Alice in Wonderland”
2. Normalize the words such that every value lies within a range of 0 and 1
3. Compute the cosine distance between the given words:
    - Alice 
    - Dinah
    - Rabbit
4. List the 5 closest words to 'Alice'. Discuss the results.
5. Discuss what the main drawbacks are of a term-term co-occurence matrix solutions?


In [52]:
################################################################################
############################## 0. Util functions ###############################
################################################################################

# Run this block first
def getKeyFromValue(dictionary, target):
    return list(dictionary.keys())[list(dictionary.values()).index(target)]


In [66]:
################################################################################
################ 1a. Computing word-word co-ocurrence matrix ###################
################################################################################

# Not changing V in the cell they do not want us to modify, but one never uses a 
# size variable to handle an iteration issue.
#V = len(tokenizer.word_index)

# Create my co-ocurrence matrix, initially 0 (VxV size)
# How to access this matrix:
#   Each column is an (index - 1) (column 0 is word in index 1; column 1 is word in index 2; ...)
wcoMatrix = np.zeros([V, V]) 

# Using my window_size_corpus to define my context scope
scope = window_size_corpus

# TODO: Do not make that many iterations
# Greedy approach first (to be able to compare the optimization)

# For each line in the corpus. Note that they preserve the order, even when they are indexes now.
for s in corpus:
    # For each word
    for current_index in range(0, len(s)):
        current_value = s[current_index] # Get the 'word'
        # From left to right
        for neighbor_index in range(current_index - scope, current_index + scope + 1):
            # Never out of boundaries and never the same index
            if ( neighbor_index >= 0 and neighbor_index < len(s) ) and ( neighbor_index != current_index ):
                # Get my neighbor 'word'
                neighbor = s[neighbor_index] 
                # Never myself and myself (keeping diagonal to 0)
                if current_value != neighbor:
                    # Update the ocurrence
                    #wcoMatrix[current_value - 1, neighbor - 1] += 1
                    wcoMatrix[current_value, neighbor] += 1
                    
# Finally normalizing
wcoMatrix = wcoMatrix / wcoMatrix.max()
wcoMatrix

array([[0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.93421053, ..., 0.01315789, 0.        ,
        0.        ],
       [0.        , 0.93421053, 0.        , ..., 0.01315789, 0.01315789,
        0.01315789],
       ...,
       [0.        , 0.01315789, 0.01315789, ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.01315789, ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.01315789, ..., 0.        , 0.        ,
        0.        ]])

In [67]:
################################################################################
############ 1b. Find cosine similarity to Alice, Dinah and Rabbit #############
################################################################################

words_to_compare = ["Alice", "Dinah", "Rabbit"]

# Get my index
full_index = tokenizer.word_index

# Iterate on all possible combinations
for w1 in words_to_compare:
    w1 = w1.lower() 
    x = full_index[w1]
    for w2 in words_to_compare:
        w2 = w2.lower()
        if w1 != w2:
            y = full_index[w2]
            X = wcoMatrix[x, :].reshape((1, V))
            Y = wcoMatrix[y, :].reshape((1, V))
            print("cosine_similarity(%s, %s)=%s" % (w1, w2, cosine_similarity(X, Y)))

cosine_similarity(alice, dinah)=[[0.39360011]]
cosine_similarity(alice, rabbit)=[[0.47890931]]
cosine_similarity(dinah, alice)=[[0.39360011]]
cosine_similarity(dinah, rabbit)=[[0.29862324]]
cosine_similarity(rabbit, alice)=[[0.47890931]]
cosine_similarity(rabbit, dinah)=[[0.29862324]]


In [68]:
################################################################################
##################### 1c. Find the closest words to Alice #######################
################################################################################

# It was not importing
from sklearn.neighbors import NearestNeighbors

# Define our variables
targetWord = "Alice".lower()
X = wcoMatrix
N_NEIGHBORS = 5
full_index = tokenizer.word_index

# Perfrom the NearestNeibhbors algorithm
# TODO: Tune parameters
nbrs = NearestNeighbors(n_neighbors = N_NEIGHBORS, algorithm = 'ball_tree').fit(X)

# Get the index corresponding to the target word
aliceMatrixIndex = full_index[targetWord]

# Get results from nearest neighbors
distances, indices = nbrs.kneighbors(X)
aliceIndices = indices[aliceMatrixIndex, :]
aliceDistances = distances[aliceMatrixIndex, :]

# Output result
print("Nearest neighbors to %s (index %s):" % (targetWord, aliceMatrixIndex))
for i in range(0, N_NEIGHBORS):
    print( "%s [%s] with distance: %s" % (getKeyFromValue(full_index, aliceIndices[i]), aliceIndices[i], aliceDistances[i]) )


Nearest neighbors to alice (index 11):
alice [11] with distance: 0.0
her [14] with distance: 0.5616433150108668
that [13] with distance: 0.5692975698101006
herself [41] with distance: 0.5700573426956416
for [18] with distance: 0.5811858638252018


**Discussion of the drawbacks**

TODO

In [56]:
#Save your all the vector representations of your word embeddings in this way
#Change when necessary the sizes of the vocabulary/embedding dimension

f = open('vectors_co_occurrence.txt',"w")
f.write(" ".join([str(V-1),str(V-1)]))
f.write("\n")

#vectors = your word co-occurrence matrix
vectors = wcoMatrix
for word, i in tokenizer.word_index.items():    
    f.write(word)
    f.write(" ")
    f.write(" ".join(map(str, list(vectors[i,:]))))
    f.write("\n")
f.close()

In [ ]:
#reopen your file as follows

co_occurrence = KeyedVectors.load_word2vec_format('./vectors_co_occurrence.txt', binary=False)

## Question 2

### Word embeddings
Build embeddings with a keras implementation where the embedding vector is of length 50, 150 and 300. Use the Alice in Wonderland text book for training.
1. Using the CBOW model
2. Using Skipgram model
3. Add extra hidden dense layer to CBow and Skipgram implementations. Choose an activation function for that layer and justify your answer.
4. Analyze the four different word embeddings
    - Implement your own function to perform the analogy task with. Do not use existing libraries for this task such as Gensim. Your function should be able to answer whether an anaology as in the example given in the pdf-file is true.
    - Compare the performance on the analogy task between the word embeddings that you have trained in 2.1, 2.2 and 2.3.  
    - Visualize your results and interpret your results
5. Use the word co-occurence matrix from Question 1. Compare the performance on the analogy task with the performance of your trained word embeddings.  
6. Discuss:
    - What are the main advantages of CBOW and Skipgram?
    - What is the advantage of negative sampling?
    - What are the main drawbacks of CBOW and Skipgram?
7. Load pre-trained embeddings on large corpuses (see the pdf file). You only have to consider the word embeddings with an embedding size of 300
    - Compare performance on the analogy task with your own trained embeddings from "Alice in Wonderland". You can limit yourself to the vocabulary of Alice in Wonderland. Visualize the pre-trained word embeddings and compare these with the results of your own trained word embeddings. 


In [ ]:
#prepare data for cbow


In [ ]:
#create CBOW model


In [ ]:
#define loss function


In [ ]:
#train model


In [ ]:
#prepare data for Skipgram


In [ ]:
#create Skipgram model


In [ ]:
#define loss function for Skipgram


In [ ]:
#train Skipgram model


In [ ]:
#create CBOW model with additional dense layer


In [ ]:
#define loss function for CBOW + dense


In [ ]:
#train model for CBOW + dense


In [ ]:
#create Skipgram with additional dense layer


In [ ]:
#define loss function for Skipgram + dense


In [ ]:
#train model for Skipgram + dense


In [ ]:
#Implement your own analogy function


Comparison performance:

In [ ]:
#Visualization results trained word embeddings


Interpretation results of the visualization

Compare the results of the trained word embeddings with the word-word co-occurrence matrix

Discussion of the advantages of CBOW and Skipgram, the advantages of negative sampling and drawbacks of CBOW and Skipgram

In [ ]:
#load pretrained word embeddings of word2vec

path_word2vec = "your path /GoogleNews-vectors-negative300.bin"

word2vec = KeyedVectors.load_word2vec_format(path, binary=True)

In [ ]:
#load pretraind word embeddings of Glove

path = "your path /glove.6B/glove.6B.300d_converted.txt"

#convert GloVe into word2vec format
gensim.scripts.glove2word2vec.get_glove_info(path)
gensim.scripts.glove2word2vec.glove2word2vec(path, "glove_converted.txt")

glove = KeyedVectors.load_word2vec_format(path, binary=False)

In [ ]:
#Visualize the pre-trained word embeddings

Comparison performance with your own trained word embeddings